## Script to check quality of segmentation masks

In [1]:
import os
import numpy as np
# Visualization
import matplotlib.pyplot as plt

import tifffile as tiff
from preprocessing import (filter_segmentation, get_image_paths,
                           run_segmentation, SEGMENTATION_CONFIG)

In [2]:
IMG_DIR = "/mnt/imaging.data/PertzLab/apoDetection/TIFFs"
MASK_DIR = "/home/nbahou/myimaging/apoDet/data/dataset1/apo_masks"

image_paths = get_image_paths(os.path.join(IMG_DIR))
filenames = [os.path.splitext(os.path.basename(path))[0]
             for path in image_paths[:2]]

paths = [
        os.path.abspath(os.path.join(MASK_DIR, f))
        for f in os.listdir(MASK_DIR)
    ]
mask_paths = sorted(paths)
mask_filenames = [os.path.splitext(os.path.basename(path))[0]
             for path in mask_paths[:2]]

index = 0
masks = np.load(mask_paths[index])['gt']
imgs = tiff.imread(image_paths[index])


In [11]:
print(mask_filenames)

['Exp01_Site01', 'Exp01_Site02']


In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output

n_frames, H, W = imgs.shape

# --- Widgets ---
frame_slider = widgets.IntSlider(0, 0, n_frames-1, 1, description="Frame")
x_slider = widgets.IntSlider(0, 0, W-1, 10, description="X")
y_slider = widgets.IntSlider(0, 0, H-1, 10, description="Y")
window_slider = widgets.IntSlider(200, 10, min(H, W), 10, description="Window size")
invert_checkbox = widgets.Checkbox(value=False, description='Invert image')
play_button = widgets.Play(interval=150, value=0, min=0, max=n_frames-1, step=1, description="Play")
widgets.jslink((play_button, 'value'), (frame_slider, 'value'))

prev_button = widgets.Button(description="Previous")
next_button = widgets.Button(description="Next")
out = widgets.Output()

current_frame = 0

# --- Display Function ---
def show_frame(frame):
    x = x_slider.value
    y = y_slider.value
    size = window_slider.value

    x_end = min(x + size, W)
    y_end = min(y + size, H)

    img_crop = imgs[frame, y:y_end, x:x_end]
    mask_crop = masks[frame, y:y_end, x:x_end]
    mask_display = np.ma.masked_where(mask_crop == 0, mask_crop)

    # Invert if toggle is checked
    if invert_checkbox.value:
        img_crop = np.clip(img_crop, 0, 5000)
        img_crop = 5000 - img_crop

    with out:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(img_crop, cmap='gray', vmin=0, vmax=5000)
        ax.imshow(mask_display, cmap='jet', alpha=0.5)
        ax.set_title(f"Frame {frame}")
        ax.axis('off')
        plt.show()

# --- Event Handlers ---
def on_slider_change(change):
    global current_frame
    current_frame = frame_slider.value
    show_frame(current_frame)

def on_prev_clicked(b):
    global current_frame
    if current_frame > 0:
        current_frame -= 1
        frame_slider.value = current_frame

def on_next_clicked(b):
    global current_frame
    if current_frame < n_frames - 1:
        current_frame += 1
        frame_slider.value = current_frame

# --- Link events ---
frame_slider.observe(on_slider_change, names='value')
x_slider.observe(lambda c: show_frame(current_frame), names='value')
y_slider.observe(lambda c: show_frame(current_frame), names='value')
window_slider.observe(lambda c: show_frame(current_frame), names='value')
invert_checkbox.observe(lambda c: show_frame(current_frame), names='value')
prev_button.on_click(on_prev_clicked)
next_button.on_click(on_next_clicked)

# --- Layout and launch ---
controls = widgets.VBox([
    widgets.HBox([prev_button, frame_slider, next_button, play_button]),
    x_slider,
    y_slider,
    window_slider,
    invert_checkbox
])

display(controls, out)
show_frame(current_frame)


Output()